# Рейтинговая рекомендательная система
_______

Начнём программировать твою первую неперсонализированную рекомендательную систему. 

Наш план такой:
 - Загрузим **датасет из файла movies_metadata_fixed.csv**. 
 - Датасет находится в «сыром» виде, поэтому сделаем **предобработку данных**. Другими словами, подготовим все необходимые колонки из датасета и приведем в удобный формат.
 - Напишем функции, которые будут оценивать каждый фильм по некоторой **формуле**. Формул будет несколько,поэтому сможем сравнить разные рейтинги, оценить разницу и выбрать лучший вариант.
 
 _____

Для обработки и анализа датасета нужны библиотеки Pandas и Ast, импортируем их

In [1]:
import warnings
warnings.filterwarnings('ignore')

**Pandas** позволит загружать и трансформировать данные при помощи простых и понятных функций

In [2]:
#загрузи pandas под псевдонимом pd
import pandas as pd

Библиотека **Ast** и пакет **Literal_eval** пригодятся для извлечения информации из «сырых» форматов данных

In [3]:
from ast import literal_eval

Загружаем датасет из файла **movies_metadata_fixed.csv**

In [4]:
path_to_dataset = 'movies_metadata_fixed.csv'#укажи верный путь к файлу на своем устройстве
dataset = pd.read_csv(path_to_dataset)
dataset.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92


По строкам в датасете хранятся отдельные фильмы. После загрузки файла, ты увидишь, что в столбцах для каждого фильма доступна разная информация: бюджет, название, актеры, режиссеры, обзор, язык и т.д. 

В первом задании нас интересуют несколько переменных: жанр(genres), год выпуска(year), средний рейтинг фильма(vote_average) и количество оценок фильма(vote_count).

____

### Жанры

Начнем с колонки "жанры" и посмотрим, как она выглядит:

In [5]:
#посмотри на значения элементов в колонке genres

Что видим? Строка, в которой записан список словарей. Очевидно, что нам не нужно столько информации, поэтому преобразуем это поле в обычный список. Здесь пригодится пакет literal_eval, который может обрабатывать подобные строки. Например:

In [6]:
literal_eval(dataset['genres'].iloc[0])

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

На выходе **Literal_eval** всё тот же список словарей, но теперь он не в виде строки, а в виде объектов языка Python.

Чтобы получить список жанров из такой структуры, нужно обратиться к полю **name** в каждом элементе списка словарей.

In [7]:
[i['name'] for i in literal_eval(dataset['genres'].iloc[0])]

['Animation', 'Comedy', 'Family']

Теперь преобразуем поле **genres** целиком так же, как и в примере выше 

In [8]:
dataset['genres'] = dataset['genres'].fillna('[]')\
                                     .apply(literal_eval)\
                                     .apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [9]:
dataset['genres'].head()

0     [Animation, Comedy, Family]
1    [Adventure, Fantasy, Family]
2               [Romance, Comedy]
3        [Comedy, Drama, Romance]
4                        [Comedy]
Name: genres, dtype: object

Выясним, фильмы каких жанров содержатся в датасете. 

In [10]:
import functools
available_genres = functools.reduce(lambda x,y: set(x).union(set(y)), dataset.genres.tolist())
available_genres

{'Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Foreign',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'TV Movie',
 'Thriller',
 'War',
 'Western'}

Отлично, мы получили в распоряжение полный список жанров фильмов в датасете, переходим к следующей колонке.

___


###  Год выпуска

Год выпуска фильма можно извлечь из поля **release_date**. Сделаем это с помощью функций Pandas.

In [11]:
dataset['year'] = pd.to_datetime(dataset['release_date']).dt.year

___

### Оценки пользователей

Что дальше? Для построения рейтинга нам пригодятся 2 колонки с оценками пользователей: vote_average и vote_count. Убедимся, что эти колонки в числовом формате, а если нет, преобразуем их.

In [12]:
dataset[['vote_count', 'vote_average']].dtypes

vote_count        int64
vote_average    float64
dtype: object

Их типы int64 и float64. Все в порядке.

____

### Формула ранжирования

Перед тем как перейти к созданию формул, оставим только необходимые для работы поля в датасете, а другие удалим за ненадобностью.

In [13]:
def process_dataset(df): 
    # Удаление фильмов, для которых отсутствует информация о среднем рейтинге и кол-ве проголосовавших пользователей. 
    # Используй метод notnull() для интересующих колонок
    ok_rows = (dataset['vote_average'].notnull()&dataset['vote_count'].notnull())
    rank_dataset = dataset[ok_rows]
    
    # Оставим в нашем датасете только колонки: 'title', 'year', 'vote_count', 'vote_average', 'genres'
    usecols_ = ['title','genres','year','vote_average','vote_count']
    rank_dataset = rank_dataset[usecols_]
    
    return rank_dataset

In [14]:
dataset  = process_dataset(dataset.copy())
dataset.head()

,title,genres,year,vote_average,vote_count
0,Toy Story,"[Animation, Comedy, Family]",1995.0,7.7,5415
1,Jumanji,"[Adventure, Fantasy, Family]",1995.0,6.9,2413
2,Grumpier Old Men,"[Romance, Comedy]",1995.0,6.5,92
3,Waiting to Exhale,"[Comedy, Drama, Romance]",1995.0,6.1,34
4,Father of the Bride Part II,[Comedy],1995.0,5.7,173


Теперь гораздо удобнее работать с датасетом. Приступаем к разработке рекомендательной системы. 

Главный элемент для систем неперсональных рекомендаций — это формула для составления рейтинга фильма. Мы запрограммируем несколько вариантов такой формулы и отсортируем все фильмы по показателю рейтинга. 

У тебя может возникнуть вопрос:

— Если есть колонка vote_average, где посчитан средний рейтинг фильма, то почему бы просто не использовать это поле?

Действительно, это самый простой вариант (и даже посчитанный за нас). Давай посмотрим, что получится, если использовать такую схему для формирования рейтинга (ранжирования) фильмов.

In [15]:
def simple_formula(x):
    """
    Films are ranked by vote_average
    """
    R = x['vote_average']#извлекаем среднюю оценку vote_average
    return R

def recommend_simple(input_films, n_items):
    rank = input_films.copy()
    rank['formula'] = rank.apply(lambda x: simple_formula(x), axis=1)#применяем простую формулу ко всем строкам при помощи лямбда-функции
    rank.sort_values('formula', ascending=False, inplace=True)
    return rank.head(n_items).reset_index(drop=True)

recommend_simple(dataset, 10)

,title,genres,year,vote_average,vote_count,formula
0,Zig Zag Story,"[Drama, Comedy]",1983.0,10.0,1,10.0
1,Tall Story,[Comedy],1960.0,10.0,1,10.0
2,Birch Interval,[Drama],1976.0,10.0,1,10.0
3,Mad at the Moon,[],1992.0,10.0,1,10.0
4,The White Shadow,[],1924.0,10.0,1,10.0
5,Forever,[Documentary],2006.0,10.0,1,10.0
6,Chilly Scenes of Winter,"[Comedy, Drama, Romance]",1979.0,10.0,1,10.0
7,The Great Kidnapping,[Crime],1973.0,10.0,1,10.0
8,Backyard Dogs,"[Action, Comedy]",2001.0,10.0,1,10.0
9,"Oh, Bomb!","[Music, Comedy, Action, Crime]",1964.0,10.0,1,10.0


Какой получился результат? Мы получили список фильмов с самым высоким рейтингом, но легко заметить, что этот рейтинг был сформирован из оценок только одного пользователя. Думаю, ты бы не стал смотреть ничего из этого списка.

Как справится с этой незадачей? Простой способ: учесть количество средних оценок для каждого фильма.

In [16]:
def better_formula(x):
    v = float(x['vote_count'])#извлекаем количество оценивших vote_count
    R = x['vote_average']#извлекаем среднюю оценку vote_average
    return v*R#возвращаем их произведение, это самый простой способ учесть количество

def recommend_better(input_films, n_items):
    rank = input_films.copy()
    rank['formula'] = rank.apply(lambda x: better_formula(x), axis=1)#применим формулу ко всем строкам при помощи лямбда-функции
    rank.sort_values('formula', ascending=False, inplace=True)
    return rank.head(n_items).reset_index(drop=True)

recommend_better(dataset, 10)

,title,genres,year,vote_average,vote_count,formula
0,Inception,"[Action, Thriller, Science Fiction, Mystery, A...",2010.0,8.1,14075,114007.5
1,The Dark Knight,"[Drama, Action, Crime, Thriller]",2008.0,8.3,12269,101832.7
2,Interstellar,"[Adventure, Drama, Science Fiction]",2014.0,8.1,11187,90614.7
3,The Avengers,"[Science Fiction, Action, Adventure]",2012.0,7.4,12000,88800.0
4,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",2009.0,7.2,12114,87220.8
5,Deadpool,"[Action, Adventure, Comedy]",2016.0,7.4,11444,84685.6
6,Fight Club,[Drama],1999.0,8.3,9678,80327.4
7,Django Unchained,"[Drama, Western]",2012.0,7.8,10297,80316.6
8,Guardians of the Galaxy,"[Action, Science Fiction, Adventure]",2014.0,7.9,10014,79110.6
9,Pulp Fiction,"[Thriller, Crime]",1994.0,8.3,8670,71961.0


Видим, что результат стал гораздо лучше, в топе рейтинга находятся довольно известные и популярные фильмы. 
___

Можно ли сделать лучше? Точно, **ДА**. 

До этого мы готовы были порекомендовать любой фильм доступный в датасете, даже если его рейтинг сформирован оценкой единственного человека. 

Однако, наша задача - порекомендовать один или несколько фильмов, у которых по-настоящему высокий рейтинг, даже если сейчас его оценили всего несколько человек. Каким был бы рейтинг такого фильма, если бы его посмотрело больше людей? Мы можем только предполагать, поэтому нам остается лишь вариант со средним арифметическим, потому что по статистике, это будет наилучшая оценка среднего.

Если есть отзывы о фильме и какая-то начальная средняя оценка (даже состоящая из 1-го голоса), нечестно её не учитывать.

Для нашей финальной формулы мы попробуем учесть и среднее арифметическое, и текущую оценку одновременно. Но как? Ответ: взвешенное среднее. Это такое среднее, которое учитывает важность отдельных элементов в формуле. В нашем случае, чем больше людей (vote_count) оставили оценку фильму, тем важнее эта информация для общего рейтинга. 

In [17]:
def the_best_formula(x, Q, C):
    v = float(x['vote_count'])
    R = x['vote_average']
    return (v/(v+Q)*R)+(Q/(Q+v)*C)#попробуй самостоятельно рассчитать формулу для взвешенного среднего

def recommend_best(input_films, n_items):
    rank = input_films.copy()
    
    C = rank.vote_average.mean()
    Q = rank.vote_count.quantile(0.95)
    print("Minimum number of votes: {}. Average overall rating: {}".format(int(Q),C))
    
    rank['formula'] = rank.apply(lambda x: the_best_formula(x, Q, C), axis=1)#применим лучшую формулу ко всем строкам при помощи лямбда-функции
    rank.sort_values('formula', ascending=False, inplace=True)
    return rank.head(n_items).reset_index(drop=True)

recommend_best(dataset, 10)

Minimum number of votes: 433. Average overall rating: 5.618217011635541


,title,genres,year,vote_average,vote_count,formula
0,The Shawshank Redemption,"[Drama, Crime]",1994.0,8.5,8358,8.357778
1,The Godfather,"[Drama, Crime]",1972.0,8.5,6024,8.306376
2,The Dark Knight,"[Drama, Action, Crime, Thriller]",2008.0,8.3,12269,8.208397
3,Fight Club,[Drama],1999.0,8.3,9678,8.184925
4,Pulp Fiction,"[Thriller, Crime]",1994.0,8.3,8670,8.172184
5,Forrest Gump,"[Comedy, Drama, Romance]",1994.0,8.2,8147,8.069450
6,Schindler's List,"[Drama, History, War]",1993.0,8.3,4436,8.061058
7,Whiplash,[Drama],2014.0,8.3,4376,8.058077
8,Spirited Away,"[Fantasy, Adventure, Animation, Family]",2001.0,8.3,3968,8.035654
9,The Empire Strikes Back,"[Adventure, Action, Science Fiction]",1980.0,8.2,5998,8.025831


Когда наша формула рейтинга уже не так плоха, пофантазируем, какие еще параметры фильма могут интересовать человека?

До этого мы подготовили колонки **Жанры** и **Год выпуска фильма**. Давайте немного изменим формулу, чтобы учитывать жанр фильма и год выпуска. Формула должна принимать дополнительный параметр **Жанр**, если фильм содержит все жанры, которые выбирает пользователь. Только тогда фильм попадает в финальный рекомендованный рейтинг.

In [18]:
def recommend_by_genre(input_films, n_items, genres):
    rank = input_films.copy()
    
    C = rank.vote_average.mean()
    Q = rank.vote_count.quantile(0.95)
    print("Minimum number of votes: {}. Average overall rating: {}".format(int(Q),C))
    rank['formula'] = rank.apply(lambda x: the_best_formula(x, Q, C), axis=1)
    rank.sort_values('formula', ascending=False, inplace=True)
    
    rank['contains_genre'] = [all( y in x for y in genres) for x in rank.genres]
    rank = rank[rank.contains_genre == True]
    rank.drop('contains_genre', axis=1, inplace=True)

    return rank.head(n_items).reset_index(drop=True)

In [19]:
recommend_by_genre(dataset, 10, genres=['Adventure', 'Science Fiction'])

Minimum number of votes: 433. Average overall rating: 5.618217011635541


,title,genres,year,vote_average,vote_count,formula
0,The Empire Strikes Back,"[Adventure, Action, Science Fiction]",1980.0,8.2,5998,8.025831
1,Inception,"[Action, Thriller, Science Fiction, Mystery, A...",2010.0,8.1,14075,8.025780
2,Interstellar,"[Adventure, Drama, Science Fiction]",2014.0,8.1,11187,8.007335
3,Star Wars,"[Adventure, Action, Science Fiction]",1977.0,8.1,6778,7.950685
4,Back to the Future,"[Adventure, Comedy, Science Fiction, Family]",1985.0,8.0,6239,7.845126
5,Guardians of the Galaxy,"[Action, Science Fiction, Adventure]",2014.0,7.9,10014,7.805238
6,Return of the Jedi,"[Adventure, Action, Science Fiction]",1983.0,7.9,4763,7.709489
7,2001: A Space Odyssey,"[Science Fiction, Mystery, Adventure]",1968.0,7.9,3075,7.617842
8,The Martian,"[Drama, Adventure, Science Fiction]",2015.0,7.6,7442,7.490819
9,Captain America: The Winter Soldier,"[Action, Adventure, Science Fiction]",2014.0,7.6,5881,7.463831


Супер, теперь мы можем выбирать произвольные жанры фильмов.

То же самое сделаем с параметром **год выпуска фильма**. У формулы появляется ещё один параметр, если фильм был выпущен в указанный год или позже, тогда он попадает в финальную выдачу.

In [20]:
def recommend_by_genre_and_year(input_films, n_items, min_votes, genres, not_older_than):
    
    #доработай предыдущую функцию, добавив условное ограничение по году
    rank = input_films.copy()
    
    C = rank.vote_average.mean()
    Q = rank.vote_count.quantile(0.95)
    print("Minimum number of votes: {}. Average overall rating: {}".format(int(Q),C))
    rank['formula'] = rank.apply(lambda x: the_best_formula(x, Q, C), axis=1)
    rank.sort_values('formula', ascending=False, inplace=True)
    
    rank['contains_genre'] = [all( y in x for y in genres) for x in rank.genres]
    rank = rank[rank.contains_genre == True]
    rank.drop('contains_genre', axis=1, inplace=True)
    rank = rank[rank.year >= not_older_than]
    
    return rank.head(n_items).reset_index(drop=True)

In [21]:
recommend_by_genre_and_year(dataset, 15, 150, genres=['Adventure', 'Science Fiction'], not_older_than=2015)

Minimum number of votes: 433. Average overall rating: 5.618217011635541


,title,genres,year,vote_average,vote_count,formula
0,The Martian,"[Drama, Adventure, Science Fiction]",2015.0,7.6,7442,7.490819
1,Guardians of the Galaxy Vol. 2,"[Action, Adventure, Comedy, Science Fiction]",2017.0,7.6,4858,7.437507
2,Star Wars: The Force Awakens,"[Action, Adventure, Science Fiction, Fantasy]",2015.0,7.5,7993,7.403107
3,Rogue One: A Star Wars Story,"[Action, Adventure, Science Fiction]",2016.0,7.4,5111,7.260572
4,Mad Max: Fury Road,"[Action, Adventure, Science Fiction, Thriller]",2015.0,7.3,9629,7.227484
5,Avengers: Age of Ultron,"[Action, Adventure, Science Fiction]",2015.0,7.3,6908,7.200608
6,Captain America: Civil War,"[Adventure, Action, Science Fiction]",2016.0,7.1,7462,7.018572
7,Doctor Strange,"[Action, Adventure, Fantasy, Science Fiction]",2016.0,7.1,5880,6.998170
8,Okja,"[Adventure, Drama, Fantasy, Science Fiction]",2017.0,7.7,795,6.964964
9,Ant-Man,"[Science Fiction, Action, Adventure]",2015.0,7.0,6029,6.907231


По аналогии ты можешь добавлять любые другие параметры в свою рекомендательную систему.
____

# Итоги

Ты разработал простую рекомендательную систему. Она строится на предположении, что фильмы, которые нравятся большинству, понравятся и тебе. Конечно, это не совсем так, поэтому в следующих заданиях мы учтём персональные вкусы и запрограммируем более интеллектуальные рекомендательные системы.